<a href="https://colab.research.google.com/github/riyakinnarkar/greenlink/blob/main/green_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install PuLP
!pip install pulp --quiet
from pulp import *
import matplotlib.pyplot as plt

# ======= DEMO DATA =======
use_demo = input("Use demo dataset? (yes/no): ").lower()
if use_demo == "yes":
    # Demo dataset for a coffee shop
    costs = [5, 6, 4.5]
    emissions = [2, 1.8, 2.5]
    capacities = [100, 80, 120]
    demand = 150
    budget = 700
    num_suppliers = len(costs)
else:
    # Ask how many suppliers
    num_suppliers = int(input("How many suppliers do you have? "))

    costs = []
    emissions = []
    capacities = []

    for i in range(num_suppliers):
        print(f"\nSupplier {i+1}:")
        cost = float(input("  Cost per unit ($): "))
        emission = float(input("  Emissions per unit (kg CO2): "))
        cap = float(input("  Maximum capacity (units): "))
        costs.append(cost)
        emissions.append(emission)
        capacities.append(cap)

    # Ask for total demand & budget
    demand = float(input("\nEnter your total demand (units): "))
    budget = float(input("Enter your total budget ($): "))

# ======= BUILD LP MODEL =======
model = LpProblem("SustainableSupplyChain", LpMinimize)

# Decision variables
x = [LpVariable(f"x{i}", lowBound=0) for i in range(num_suppliers)]

# Multi-objective: weighted emissions + cost
weight_emissions = 0.7
weight_cost = 0.3
model += lpSum([weight_emissions*emissions[i]*x[i] + weight_cost*costs[i]*x[i]
                for i in range(num_suppliers)]), "WeightedObjective"

# Constraints
model += lpSum([x[i] for i in range(num_suppliers)]) == demand, "Demand"
model += lpSum([costs[i]*x[i] for i in range(num_suppliers)]) <= budget, "Budget"
for i in range(num_suppliers):
    model += x[i] <= capacities[i], f"Capacity_{i}"

# ======= SCENARIO ANALYSIS =======
scenarios = {
    "Normal": (demand, budget, capacities),
    "Budget cut": (demand, budget*0.8, capacities),
    "Supplier 2 down": (demand, budget, [capacities[0], 0, capacities[2]] if num_suppliers >=3 else capacities),
}

for name, (d, b, caps) in scenarios.items():
    # Update constraints
    for i in range(num_suppliers):
        model.constraints[f"Capacity_{i}"].changeRHS(caps[i])
    model.constraints["Demand"].changeRHS(d)
    model.constraints["Budget"].changeRHS(b)

    # Solve
    model.solve()

    # Show results
    print(f"\n===== Scenario: {name} =====")
    for i in range(num_suppliers):
        print(f" Supplier {i+1}: {x[i].varValue:.2f} units")
    print("Total weighted objective (emissions+cost):", value(model.objective))

    # Plot emissions per supplier
    emissions_per_supplier = [emissions[i]*x[i].varValue for i in range(num_suppliers)]
    plt.bar([f"Supplier {i+1}" for i in range(num_suppliers)], emissions_per_supplier)
    plt.ylabel("Emissions (kg CO2)")
    plt.title(f"Emissions per Supplier - {name}")
    plt.show()
